In [50]:
!pip install geopy


In [1]:
import warnings
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
import os
import pygame
from gtts import gTTS
from time import sleep
import speech_recognition as sr
import tempfile
import torch
import datetime  # New import for date and time
import requests  # For accessing location service

pygame 2.6.0 (SDL 2.28.4, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Suppress FutureWarning for tokenization spaces
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")
# Load the model and tokenizer globally to avoid reloading every time
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)


In [3]:
# OpenWeatherMap API key
WEATHER_API_KEY = '6375f15162952d140f9908db1d7ffb99'


In [4]:
def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    recognizer.energy_threshold = 300  # Lower energy threshold for faster detection
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening for your query...")
        recognizer.adjust_for_ambient_noise(source, duration=0.5)  # Optimized ambient noise detection
        
        try:
            # Listen with optimized timeout and phrase time limit
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)  # Reduced timeout and phrase limit

        except sr.WaitTimeoutError:
            print("Listening timed out, please speak again.")
            return None  # Return None if timeout occurs

    try:
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")  # Display user input
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the Google Speech Recognition service; {e}")
        return None


In [5]:
def handle_time_date_queries(user_input):
    if "time" in user_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return f"The current time is {current_time}."
    elif "date" in user_input or "today" in user_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return f"Today is {current_date}."
    elif "location" in user_input or "where am i" in user_input:
        return get_current_location()  # Fetch current location
    elif "weather" in user_input or "weather in my current location" in user_input:
        return get_current_weather()
    else:
        return None


In [6]:
def select_microphone():
    mic_list = sr.Microphone.list_microphone_names()
    index = 2
    # Removed print statement for microphone selection
    return index


In [7]:
def get_current_location():
    try:
        # Using a free IP geolocation service
        response = requests.get("http://ip-api.com/json")
        data = response.json()
        
        if data['status'] == 'success':
            city = data.get('city', 'unknown city')
            region = data.get('regionName', 'unknown region')
            country = data.get('country', 'unknown country')
            return f"You are currently in {city}, {region}, {country}."
        else:
            return "Sorry, I could not determine your location."
    except Exception as e:
        return f"Error fetching location: {str(e)}"


In [8]:
def get_current_weather():
    location = get_current_location()
    if "in" in location:
        city = location.split("in ")[1].split(",")[0]
        try:
            weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
            response = requests.get(weather_url)
            data = response.json()
            if data['cod'] == 200:
                weather_description = data['weather'][0]['description']
                temperature = data['main']['temp']
                return f"The current weather in {city} is {weather_description} with a temperature of {temperature}°C."
            else:
                return "Sorry, I couldn't get the weather information."
        except Exception:
            return "Error fetching weather."
    else:
        return "I couldn't determine the location for weather."

In [9]:
def speak_response(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        tts.save(temp_audio_file.name)
        temp_file_path = temp_audio_file.name
    pygame.mixer.init()
    try:
        pygame.mixer.music.load(temp_file_path)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.wait(50)  # Check every 50ms for faster termination

    except Exception as e:
        print(f"Error playing audio: {e}")
    finally:
        pygame.mixer.quit()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)


In [10]:
def generate_response(user_input):
    # Check if the user asked for the AI's name
    if "what is your name" in user_input.lower():
        return "I am Amigo, your all-weather conversational companion, just like the bike that you are riding on."
    
    # Handle time and date queries separately
    time_date_response = handle_time_date_queries(user_input)
    if time_date_response:
        return time_date_response

    # Check for factual or current event queries
    if any(keyword in user_input.lower() for keyword in [ "who is", "when is", "where is", "current", "today", "news","capital","Places"]):
        return "Thank you for your query, but that's out of my scope of training."

    # Tokenize user input
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True, max_length=50)  # Reduced max length
    attention_mask = inputs['attention_mask']

    # Ensure the attention mask is correctly set
    if 'attention_mask' not in inputs:
        inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])  # Create a default attention mask
    
    # Generate bot response with reduced max output length
    reply_ids = model.generate(inputs['input_ids'], max_length=50, pad_token_id=tokenizer.eos_token_id)  # Reduced response length
    
    # Decode response with clean-up for tokenization spaces
    bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True, attention_mask=attention_mask, clean_up_tokenization_spaces=True)
    
    return bot_response


In [11]:
def listen_for_wake_word(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)

    print("Waiting for wake word 'Hey Amigo'...")
    while True:
        with mic as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)  # Faster ambient noise adjustment
            audio = recognizer.listen(source)  # Removed timeout for seamless experience
            
        try:
            user_input = recognizer.recognize_google(audio)
            print(f"User said: {user_input}")
            if "hey amigo" in user_input.lower():
                print("Wake word detected!")  # Display wake word detected immediately
                return True  # Wake word detected, start conversation
        except sr.UnknownValueError:
            continue
        except sr.RequestError as e:
            print(f"Error with the Google Speech Recognition service: {e}")
            continue


In [12]:
def amigo_conversational_companion():
    mic_index = select_microphone()

    if listen_for_wake_word(mic_index):
        greeting = "Hi Hello Namaskara, I am Amigo. How can I assist you today?"
        print(f"Amigo: {greeting}")
        speak_response(greeting)

        while True:
            user_input = capture_speech(mic_index)
            if user_input is None:
                continue

            if "exit" in user_input.lower() or "goodbye" in user_input.lower():
                farewell_message = "Goodbye, have a nice day!"
                print(f"Amigo: {farewell_message}")
                speak_response(farewell_message)
                break

            bot_response = generate_response(user_input)
            print(f"Amigo: {bot_response}")
            speak_response(bot_response)


In [17]:
if __name__ == "__main__":
    amigo_conversational_companion()


Waiting for wake word 'Hey Amigo'...
User said: hey Amigo
Wake word detected!
Amigo: Hi Hello Namaskara, I am Amigo. How can I assist you today?
Listening for your query...
User said: what is the weather in my current location
Amigo: You are currently in Bengaluru, Karnataka, India.
Listening for your query...
User said: what is the weather in my location
Amigo: You are currently in Bengaluru, Karnataka, India.
Listening for your query...
User said: what is the weather in my place
Amigo: The current weather in Bengaluru is scattered clouds with a temperature of 28.54°C.
Listening for your query...
User said: exit
Amigo: Goodbye, have a nice day!
